# **Sports Image Classification**

# I. Dataset Collection

In [ ]:
#!pip install -q kaggle #As the dataset is in the kaggle this command helps installing the API of kaggle


Dataset Collection

Installing the Necessary Libraries

Dataset Overview

Visualizations

Data Cleaning and Preprocessing

Modelling Techniques

Comparison of Model Perfrmances

Predections on New Data



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!cp /content/gdrive/MyDrive/path/to/kaggle.json /root/.kaggle/kaggle.json


In [ ]:
#from google.colab import files

#Upload Kaggle API key
#uploaded = files.upload()
#uploaded = 

# import json
# with open('kaggle.json') as f:
#     data = json.load(f)

In [ ]:
# !mkdir -p /root/.kaggle
# !cp kaggle.json /root/.kaggle/
# #!cp /content/drive/MyDrive/Colab Notebooks/AdvancedPython -Image Classification/kaggle.json /root/.kaggle/kaggle.json
# !chmod 600 /root/.kaggle/kaggle.json


In [ ]:
# # Replace 'gpiosenka/sports-classification' with your Kaggle username and dataset name
# !kaggle datasets download -d gpiosenka/sports-classification

# # Unzip the dataset
# !unzip -q sports-classification.zip -d /content/dataset


In [ ]:

# !kaggle datasets download -d gpiosenka/sports-classification #This is the username and dataset. API Key from the Kaggle

Installing plotly for interactive plots in the following Project.

# II. Installing and Importing the Necessary Libraries

In [ ]:
pip install plotly

In [ ]:
pip install tensorflow-metal

In [ ]:
#!sudo apt-get install tesseract-ocr
!pip install pytesseract
!pip install keras-tuner

In [ ]:
pip install tensorflow


In [ ]:
pip install plotly --upgrade


In [24]:
# Importing all the necessary libraries for the project.

import os
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import pandas as pd
import plotly.express as px
import cv2
import numpy as np
import pytesseract
from io import BytesIO
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import plotly.io as pio


In [ ]:
#pio.renderers.default = 'browser'
pio.renderers.default = 'iframe_connected'

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

##  IV. Elementary Data Analysis (EDA)

**Loading images form the directories**

In [ ]:
# As the dataset is organized into train, test, and valid directories
train_path = r'/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/train'
test_path = r'/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/test'
valid_path = r'/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/valid'

# As the CSV file is in the dataset directory
csv_file_path = r'/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/sports.csv'

# List files in the directories
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)
valid_files = os.listdir(valid_path)

# Read CSV file
df = pd.read_csv(csv_file_path)

# Display basic information about the dataset
print("Number of training samples:", len(train_files))
print("Number of test samples:", len(test_files))
print("Number of validation samples:", len(valid_files))

# Display the first few rows of the CSV file
print("\nCSV file content:")
print(df.head())


The data visualization.

This code defines a Python function called visualize_individual_samples that, given a dataset path, displays a specified number of individual images from each class in the dataset using Matplotlib. The function iterates through each class, selects a few samples, opens and resizes the images, and then uses Matplotlib to show each image with its corresponding class name as the title. This visualization aids in exploring and understanding the dataset during the exploratory data analysis (EDA) phase of a machine learning project. The function is called with the training set path and a specified image size for display.


In [ ]:
# Function to visualize each image with the class name and dimensions
def visualize_individual_samples_with_dimensions(data_path, num_samples_per_class=3, image_size=(64, 64)):
    classes = os.listdir(data_path)

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        class_samples = os.listdir(class_path)[:num_samples_per_class]

        for sample in class_samples:
            sample_path = os.path.join(class_path, sample)

            # Use matplotlib to display the image
            img = Image.open(sample_path)
            img = img.resize(image_size)  # Resize the image to the specified size

            # Get the dimensions of the image
            original_width, original_height = img.size

            # Create a figure with two subplots
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

            # Plot the original image
            ax1.imshow(img)
            ax1.set_title(f"Class: {class_name}")
            ax1.axis('off')

            # Plot the dimensions separately
            ax2.text(0.5, 0.5, f"Dimensions: {original_width} x {original_height}", ha='center', va='center', fontsize=14)
            ax2.axis('off')

            plt.show()

# Visualize each image from the training set with the class name and dimensions
visualize_individual_samples_with_dimensions(train_path, image_size=(128, 128))


The following Python funtions visualize_class_distribution, visualize_class_distribution_pie, visualize_class_distribution_treemap, visualize_class_distribution_scatter use Plotly Express to create an interactive bar plot, pie chart, treemap and scatter plot respeectively showcasing the distribution of samples across different classes in a dataset. These functions takes the dataset path as input, computes the number of samples per class, and generates plots with custom colors and layout settings. The resulting visualization provides insights into the distribution of data, with each unit representing a class and its size indicating the number of samples.

In [ ]:
#Function to visualise the distribution on the trainingset as the bar graph.
def visualize_class_distribution(data_path):
    classes = os.listdir(data_path)

    # Filter out non-directory entries (e.g., .DS_Store on macOS)
    classes = [class_name for class_name in classes if os.path.isdir(os.path.join(data_path, class_name))]

    class_distribution = [len(os.listdir(os.path.join(data_path, class_name))) for class_name in classes]

    # Create an interactive bar plot using Plotly Express
    fig = px.bar(x=classes, y=class_distribution, labels={'x': 'Class', 'y': 'Number of Samples'},
                 title='Class Distribution in Training Set', width=1200, height=900,
                 color=classes, color_continuous_scale='Viridis')

    # Customize the layout for a professional look
    fig.update_layout(
        xaxis_title='Class',
        yaxis_title='Number of Samples',
        showlegend=False,  # Hide the legend for simplicity
        margin=dict(l=50, r=50, t=50, b=50),  # Adjust margin for better spacing
        plot_bgcolor='lightgray',  # Set plot background color to light gray
        paper_bgcolor='white',  # Set paper background color to white
        xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='lightgray'),  # Show x-axis grid
        yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='lightgray'),  # Show y-axis grid
    )

    # Show the plot interactively
    fig.show()

# Visualize class distribution interactively in the training set
visualize_class_distribution(train_path)


The above plot 'Class distribution in Training Set' shows that the number of samples in the training dataset varies from the number 59 to 191. The 59 image samples are for the class sky surfing and the 191 image samples are for the football.

In [ ]:
#Function to visualise the distribution on the trainingset as the pie chart.
def visualize_class_distribution_pie(data_path):
    # Get class names from the subdirectories in the data path
    classes = [class_name for class_name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, class_name))]

    # Calculate the number of samples for each class
    class_distribution = [len(os.listdir(os.path.join(data_path, class_name))) for class_name in classes]

    # Create an interactive pie chart using Plotly Express
    fig = px.pie(names=classes, values=class_distribution, title='Class Distribution in Training Set using a Pie Chart')

    # Update layout for better appearance
    fig.update_layout(
        autosize=False,  # Turn off autosize for custom size
        width=800,  # Set the width of the figure
        height=600,  # Set the height of the figure
        margin=dict(l=20, r=20, t=40, b=20),  # Adjust margin for better spacing
    )

    # Update traces (slices) to improve the pie chart layout
    fig.update_traces(
        pull=[0.1] * len(classes),  # Adjust the pull to increase the size of slices
        textinfo='percent+label',  # Show percentage and label in each slice
        textposition='inside',  # Show text inside the slices
        marker=dict(line=dict(color='white', width=2)),  # Add a white border to slices
    )

    # Show the plot interactively
    fig.show()


# Visualize class distribution as a pie chart in the training set with improved layout
visualize_class_distribution_pie(train_path)


In [ ]:
# Distribution of the classes as the treemap
def visualize_class_distribution_treemap(data_path):
    # Get the list of entries (files and directories) in the data path
    entries = os.listdir(data_path)

    # Filter out non-directory entries (e.g., .DS_Store on macOS)
    classes = [entry for entry in entries if os.path.isdir(os.path.join(data_path, entry))]

    # Create a dictionary to store class distribution data
    data = {'class': [], 'count': []}

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        num_samples = len(os.listdir(class_path))

        data['class'].append(class_name)
        data['count'].append(num_samples)

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Create an interactive treemap using Plotly Express
    fig = px.treemap(df, path=['class'], values='count',
                     color='count',  # Color the treemap based on the count
                     hover_data={'count': True, 'class': False},  # Display count in hover information
                     title='Class Distribution in Training Set using a Tree Map')

    # Customize the layout for better appearance and increased figure size
    fig.update_layout(
        width=1000,  # Set the width of the figure
        height=800,  # Set the height of the figure
        margin=dict(l=20, r=20, t=40, b=20),  # Adjust margin for better spacing
    )

    # Show the plot interactively
    fig.show()


# Visualize class distribution as a treemap in the training set with added features and increased figure size
visualize_class_distribution_treemap(train_path)


In [ ]:
#Visualization of the distribution of the classes as the scatter plot.
def visualize_class_distribution_scatter(data_path):
    # Get the list of entries (files and directories) in the data path
    entries = os.listdir(data_path)

    # Filter out non-directory entries (e.g., .DS_Store on macOS)
    classes = [entry for entry in entries if os.path.isdir(os.path.join(data_path, entry))]

    # Create a dictionary to store class distribution data
    data = {'class': [], 'count': []}

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        num_samples = len(os.listdir(class_path))

        data['class'].append(class_name)
        data['count'].append(num_samples)

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Create an interactive scatter plot using Plotly Express
    fig = px.scatter(df, x='class', y='count', title='Class Distribution in Training Set using a Scatter Plot',
                     labels={'class': 'Class', 'count': 'Number of Samples'},
                     category_orders={'class': classes})  # Specify the order of categories on the x-axis

    # Show the plot interactively
    fig.show()



# Visualize class distribution as a scatter plot in the training set
visualize_class_distribution_scatter(train_path)


In [ ]:
#We have to increase more visualizations.
# We can incldue image width Vs. Height
# We can include image different aspect ration and the distribition of the image aspect ratios
# We can use the label frequency as well as the line plot and everything
# We can use different color contrast of the images as different plots maybe.

# V. Data Pre-processing


THe following are the steps which are

In [ ]:

# Function to visualize each image with the class name, grayscale version, and dimensions
def visualize_individual_samples_with_dimensions(data_path, num_samples_per_class=3, image_size=(64, 64)):
    classes = os.listdir(data_path)

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        class_samples = os.listdir(class_path)[:num_samples_per_class]

        for sample in class_samples:
            sample_path = os.path.join(class_path, sample)

            # Use OpenCV to read and convert the image to grayscale
            original_img = cv2.imread(sample_path)
            grayscale_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

            # Get the dimensions of the original image
            original_height, original_width, _ = original_img.shape

            # Resize the images
            original_img = cv2.resize(original_img, image_size)
            grayscale_img = cv2.resize(grayscale_img, image_size)

            # Plot the images with dimensions
            plt.figure(figsize=(16, 6))

            # Plot the original image on the left
            plt.subplot(1, 3, 1)
            plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
            plt.title(f"Original - Class: {class_name}")
            plt.axis('on')
            plt.text(0.5, -0.1, f"Dimensions: {original_height} x {original_width}", ha='center', va='center', transform=plt.gca().transAxes)

            # Plot the grayscale image in the middle
            plt.subplot(1, 3, 2)
            plt.imshow(grayscale_img, cmap='gray')
            plt.title("Grayscale")
            plt.axis('on')

            # Plot the dimensions separately on the right
            plt.subplot(1, 3, 3)
            plt.text(0.5, 0.5, f"Dimensions: {image_size[0]} x {image_size[1]}", ha='center', va='center', fontsize=14)
            plt.axis('off')

            plt.show()

# Visualize each image from the training set with dimensions
visualize_individual_samples_with_dimensions(train_path, image_size=(128, 128))


In [ ]:
# Function to visualize each image with the class name, edge-detected version, and dimensions
def visualize_individual_samples_with_edge_detection(data_path, num_samples_per_class=3, image_size=(64, 64)):
    classes = os.listdir(data_path)

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        class_samples = os.listdir(class_path)[:num_samples_per_class]

        for sample in class_samples:
            sample_path = os.path.join(class_path, sample)

            # Use OpenCV to read and convert the image to grayscale
            original_img = cv2.imread(sample_path)

            # Apply Canny edge detection
            edges_img = cv2.Canny(original_img, 50, 150)

            # Get the dimensions of the original image
            original_height, original_width, _ = original_img.shape

            # Resize the images
            original_img = cv2.resize(original_img, image_size)
            edges_img = cv2.resize(edges_img, image_size)

            # Create a figure with two subplots
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))

            # Plot the original image on the left
            axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f"Original - Class: {class_name}")
            axes[0].axis('off')

            # Plot the edge-detected image in the middle
            axes[1].imshow(edges_img, cmap='gray')
            axes[1].set_title("Edge Detection")
            axes[1].axis('off')

            # Show dimensions separately
            dimensions_text = f"Dimensions: {original_width} x {original_height}"
            axes[2].text(0.5, 0.5, dimensions_text, ha='center', va='center', fontsize=14)
            axes[2].axis('off')

            plt.show()

# Visualize each image from the training set with edge detection and dimensions
visualize_individual_samples_with_edge_detection(train_path, image_size=(128, 128))


In [ ]:
#Function to save and plot the images which are normalised
def normalize_images(train_path, test_path, save_train_path, save_test_path):
    # Function to normalize a single image
    def normalize_image(img):
        return img / 255.0

    def plot_images(original_img, normalized_img, dimensions_text):
        # Create a figure with two subplots
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        # Plot the original image on the left
        axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original Image")
        axes[0].axis('off')

        # Plot the normalized image on the right
        axes[1].imshow(normalized_img)
        axes[1].set_title("Normalized Image")
        axes[1].axis('off')

        # Show dimensions separately
        axes[1].text(0.5, 0.5, dimensions_text, ha='center', va='center', fontsize=12, color='white')

        plt.show()

    # Normalize and plot images for the training set
    for class_name in os.listdir(train_path):
        class_path = os.path.join(train_path, class_name)
        save_class_path = os.path.join(save_train_path, class_name)

        # Create output directory if it doesn't exist
        if not os.path.exists(save_class_path):
            os.makedirs(save_class_path)

        for sample in os.listdir(class_path):
            sample_path = os.path.join(class_path, sample)
            save_sample_path = os.path.join(save_class_path, sample)

            # Read the original image
            original_img = cv2.imread(sample_path)

            # Normalize the image
            normalized_img = normalize_image(original_img)

            # Save the normalized image
            cv2.imwrite(save_sample_path, (normalized_img * 255).astype(np.uint8))

            # Get dimensions of the original image
            original_height, original_width, _ = original_img.shape
            dimensions_text = f"Dimensions: {original_width} x {original_height}"

            # Plot the images
            plot_images(original_img, normalized_img, dimensions_text)

    # Normalize and save images for the test set
    for class_name in os.listdir(test_path):
        class_path = os.path.join(test_path, class_name)
        save_class_path = os.path.join(save_test_path, class_name)

        # Create output directory if it doesn't exist
        if not os.path.exists(save_class_path):
            os.makedirs(save_class_path)

        for sample in os.listdir(class_path):
            sample_path = os.path.join(class_path, sample)
            save_sample_path = os.path.join(save_class_path, sample)

            # Read the original image
            original_img = cv2.imread(sample_path)

            # Normalize the image
            normalized_img = normalize_image(original_img)

            # Save the normalized image
            cv2.imwrite(save_sample_path, (normalized_img * 255).astype(np.uint8))

# Specify paths for the paths for saving normalized images
save_train_path = "/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/normalisedtrain"
save_test_path = "/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/normalisedtest"

# Call the function to normalize and plot images
normalize_images(train_path, test_path, save_train_path, save_test_path)


In [ ]:
import matplotlib.pyplot as plt

def plot_images(original_img, adjusted_img, dimensions_text):
    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image on the left
    axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    # Plot the adjusted image on the right
    axes[1].imshow(cv2.cvtColor(adjusted_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Adjusted Image")
    axes[1].axis('off')

    # Show dimensions separately
    axes[1].text(0.5, 0.5, dimensions_text, ha='center', va='center', fontsize=12, color='white')

    plt.show()

# Function to adjust the contrast.
def contrast_adjustment_and_save(data_path, save_path, alpha=1.5, beta=25):
    # Function to adjust contrast of a single image
    def adjust_contrast(img, alpha, beta):
        adjusted_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        return adjusted_img

    # Apply contrast adjustment and save images
    for class_name in os.listdir(data_path):
        class_path = os.path.join(data_path, class_name)
        save_class_path = os.path.join(save_path, class_name)

        # Create output directory if it doesn't exist
        if not os.path.exists(save_class_path):
            os.makedirs(save_class_path)

        for sample in os.listdir(class_path):
            sample_path = os.path.join(class_path, sample)
            save_sample_path = os.path.join(save_class_path, sample)

            # Read the original image
            original_img = cv2.imread(sample_path)

            # Adjust contrast
            adjusted_img = adjust_contrast(original_img, alpha, beta)

            # Save the adjusted image
            cv2.imwrite(save_sample_path, adjusted_img)

            # Get dimensions of the original image
            original_height, original_width, _ = original_img.shape
            dimensions_text = f"Dimensions: {original_width} x {original_height}"

            # Plot the images
            plot_images(original_img, adjusted_img, dimensions_text)

# Specify paths for the data and the path for saving contrast-adjusted images
train_path = r'/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/train'
save_path_contrast = "/Users/sarveswararaopatchipulusu/Desktop/advanced python class/archive 2/contrast_adjusted_train"

# Call the function to adjust contrast and save images
contrast_adjustment_and_save(train_path, save_path_contrast)


In [ ]:

#This is not working!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1

# Replace 'YOUR_API_KEY' with the actual API key from OCR.space
OCR_API_KEY = 'K83379042788957'

# Function to perform OCR using OCR.space API
def perform_ocr(image):
    # Convert the image to bytes
    img_byte_array = cv2.imencode('.png', image)[1].tobytes()

    # Make a POST request to OCR.space API
    response = requests.post(
        url='https://api.ocr.space/parse/image',
        headers={
            'apikey': OCR_API_KEY,
            'Content-Type': 'application/json',
        },
        files={'file': ('image.png', img_byte_array)}
    )

    # Parse the OCR result
    ocr_result = response.json()

    # Check if there are parsed results
    parsed_results = ocr_result.get('ParsedResults', [])
    if not parsed_results:
        return "No OCR result"

    # Get the first parsed result (assuming there is at least one)
    first_result = parsed_results[0]

    # Get the parsed text
    ocr_text = first_result.get('ParsedText', '')

    return ocr_text

# Function to visualize each image with the class name, OCR text, and dimensions
def visualize_individual_samples_with_ocr(data_path, num_samples_per_class=3, image_size=(64, 64)):
    classes = os.listdir(data_path)

    for class_name in classes:
        class_path = os.path.join(data_path, class_name)
        class_samples = os.listdir(class_path)[:num_samples_per_class]

        for sample in class_samples:
            sample_path = os.path.join(class_path, sample)

            # Use OpenCV to read and convert the image to grayscale
            original_img = cv2.imread(sample_path)

            # Perform OCR on the original image
            ocr_text = perform_ocr(original_img)

            # Get the dimensions of the original image
            original_height, original_width, _ = original_img.shape

            # Resize the original image
            original_img = cv2.resize(original_img, image_size)

            # Create a figure with two subplots
            fig, axes = plt.subplots(1, 2, figsize=(12, 6))

            # Plot the original image on the left
            axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f"Original - Class: {class_name}")
            axes[0].axis('on')

            # Display OCR text and dimensions on the right
            text_and_dimensions = f"OCR Text:\n{ocr_text}\n\nDimensions: {original_width} x {original_height}"
            axes[1].text(0.5, 0.5, text_and_dimensions, ha='center', va='center', fontsize=12)
            axes[1].axis('on')

            plt.show()

# Visualize each image from the training set with OCR text and dimensions
visualize_individual_samples_with_ocr(train_path, image_size=(128, 128))


# VI. Model Building


# Model 1 - A :  CNN - Convolutional Neural Networks

The first model is CNN which is convolution neural networks. And the code for that is as follows.

** On training Set and Validation set

In [ ]:
dataset_path = train_path

# Define the image size and batch size
image_size = (224, 224)
batch_size = 32

# Use ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% for training, 20% for validation
)

# Load and split the dataset into training and validation sets
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming you have multiple classes
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Optional dropout layer for regularization
model.add(layers.Dense(100, activation='softmax'))  # Assuming you have 100 classes

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # You can adjust the number of epochs based on your dataset and training performance
    validation_data=validation_generator
)

# Save the trained model
model.save('cnn_model.h5')

# Evaluate the model on the test set if available
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(test_path, target_size=image_size, batch_size=batch_size, class_mode='categorical')
# test_loss, test_acc = model.evaluate(test_generator)
# print(f'Test Accuracy: {test_acc}')


Plotting the results with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a variable 'history' that contains the training history
history = {
    'loss': [4.4792, 4.0002, 3.6973, 3.4732, 3.2785, 3.1288, 3.0180, 2.9181, 2.8353, 2.7545],
    'accuracy': [0.0306, 0.0784, 0.1216, 0.1662, 0.1912, 0.2157, 0.2387, 0.2565, 0.2740, 0.2900],
    'val_loss': [4.1028, 3.6681, 3.4190, 3.1808, 3.0134, 2.8856, 2.8474, 2.7490, 2.6982, 2.7008],
    'val_accuracy': [0.0775, 0.1362, 0.1810, 0.2344, 0.2592, 0.2904, 0.2942, 0.3119, 0.3277, 0.3160]
}

# Plotting the training and validation loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Plotting the results with Plotly Express.

In [ ]:
import plotly.express as px

# Assuming you have a variable 'history' that contains the training history
history = {
    'loss': [4.4792, 4.0002, 3.6973, 3.4732, 3.2785, 3.1288, 3.0180, 2.9181, 2.8353, 2.7545],
    'accuracy': [0.0306, 0.0784, 0.1216, 0.1662, 0.1912, 0.2157, 0.2387, 0.2565, 0.2740, 0.2900],
    'val_loss': [4.1028, 3.6681, 3.4190, 3.1808, 3.0134, 2.8856, 2.8474, 2.7490, 2.6982, 2.7008],
    'val_accuracy': [0.0775, 0.1362, 0.1810, 0.2344, 0.2592, 0.2904, 0.2942, 0.3119, 0.3277, 0.3160]
}

# Create a DataFrame for the training history
df_history = pd.DataFrame(history)
epochs = list(range(1, len(history['loss']) + 1))
df_history['Epoch'] = epochs

# Create line charts using Plotly Express
fig_loss = px.line(df_history, x='Epoch', y=['loss', 'val_loss'], labels={'value': 'Loss'},
                   title='Training and Validation Loss')
fig_accuracy = px.line(df_history, x='Epoch', y=['accuracy', 'val_accuracy'], labels={'value': 'Accuracy'},
                       title='Training and Validation Accuracy')

# Show the plots interactively
fig_loss.show()
fig_accuracy.show()


Evaluation of the model on the test set.

In [ ]:
# Evaluate the model on the test set.
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_path, target_size=image_size, batch_size=batch_size, class_mode='categorical')
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')

Plotting the model accuracy with plotly express

In [ ]:
import plotly.express as px

# Assuming you have the test accuracy value
test_accuracy = 0.3840000033378601

# Create a DataFrame for the test accuracy
df_test_accuracy = pd.DataFrame({'Metric': ['Test Accuracy'], 'Value': [test_accuracy]})

# Create a bar chart using Plotly Express
fig_test_accuracy = px.bar(df_test_accuracy, x='Metric', y='Value', text='Value',
                           title='Test Accuracy',
                           labels={'Metric': 'Metric', 'Value': 'Value'})

# Customize the layout for better appearance
fig_test_accuracy.update_layout(
    yaxis=dict(title='Accuracy'),
    showlegend=False,
    height=400,
    width=500
)

# Show the plot interactively
fig_test_accuracy.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have the test_generator and model
# test_generator = test_datagen.flow_from_directory(test_path, target_size=image_size, batch_size=batch_size, class_mode='categorical')

# Generate predictions
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

# Calculate confusion matrix for each class separately
class_names = list(test_generator.class_indices.keys())
for i in range(len(class_names)):
    class_name = class_names[i]
    
    class_true_labels = [1 if true_label == i else 0 for true_label in true_labels]
    class_predicted_labels = [1 if predicted_label == i else 0 for predicted_label in predicted_labels]
    
    conf_mat = confusion_matrix(class_true_labels, class_predicted_labels)
    
    # Plot confusion matrix using seaborn
    plt.figure(figsize=(5, 4))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Not ' + class_name, class_name],
                yticklabels=['Not ' + class_name, class_name])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for Class: {class_name}')
    plt.show()

# Display sample images with true and predicted labels
num_samples = 5
sample_indices = np.random.choice(len(test_generator.filepaths), num_samples, replace=False)

plt.figure(figsize=(15, 5))
for i, idx in enumerate(sample_indices):
    img_path = test_generator.filepaths[idx]
    img = plt.imread(img_path)
    true_label = test_generator.classes[idx]
    predicted_label = predicted_labels[idx]

    plt.subplot(1, num_samples, i + 1)
    plt.imshow(img)
    plt.title(f'True: {test_generator.class_indices.keys()} | Predicted: {test_generator.class_indices.keys()}')
    plt.axis('off')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Assuming you have the test_generator and model
# test_generator = test_datagen.flow_from_directory(test_path, target_size=image_size, batch_size=batch_size, class_mode='categorical')

# Generate predictions
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

# Map class indices to class labels
class_labels = list(test_generator.class_indices.keys())

# Display all images with true and predicted labels
num_samples = len(test_generator.filepaths)
num_columns = 5  # You can adjust the number of columns

num_rows = (num_samples + num_columns - 1) // num_columns
plt.figure(figsize=(15, 3 * num_rows))

gs = gridspec.GridSpec(num_rows, num_columns, wspace=0.05, hspace=0.3)

for i, idx in enumerate(range(num_samples)):
    img_path = test_generator.filepaths[idx]
    img = plt.imread(img_path)
    true_label = class_labels[true_labels[idx]]
    predicted_label = class_labels[predicted_labels[idx]]

    ax = plt.subplot(gs[i])
    ax.imshow(img)
    ax.set_title(f'T: {true_label}\nP: {predicted_label}', fontsize=8)
    ax.axis('off')

plt.show()


# Model 1 - B: CNN Using Hyper-paramter Tuning

Hyper parameter tuning in CNN.

Number of Filters and Kernel Size for Convolutional Layers:

Tuned Parameters:
conv1_filters: Number of filters in the first convolutional layer (tuned between 32, 64, 96, 128).
conv1_kernel: Kernel size for the first convolutional layer (tuned between 3x3 and 5x5).
conv2_filters: Number of filters in the second convolutional layer (tuned between 64, 128, 192, 256).
conv2_kernel: Kernel size for the second convolutional layer (tuned between 3x3 and 5x5).

Reasoning:
The number of filters in convolutional layers determines the complexity and expressiveness of the learned features.
The kernel size influences the receptive field and the type of features captured by each filter.

Number of Neurons in Dense (Fully Connected) Layer:

Tuned Parameter:
dense_units: Number of neurons in the dense layer (tuned between 128, 256, 384, 512).

Reasoning:
The number of neurons in the dense layer affects the capacity of the network to capture high-level abstractions.

Learning Rate:

Tuned Parameter:
learning_rate: Learning rate for the Adam optimizer (tuned between 1e-2, 1e-3, 1e-4).

Reasoning:
Learning rate controls the step size during optimization and influences the convergence and stability of the training process.

In [ ]:
# Define the dimensions of input images
img_height = 128
img_width = 128
img_channels = 3  # Assuming RGB images

# Define the number of classes in your dataset
num_classes = 100  # Adjust based on your dataset


def build_model(hp):
    model = keras.Sequential()

    model.add(layers.Conv2D(
        filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(img_height, img_width, img_channels)
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(
        filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=64),
        kernel_size=hp.Choice('conv2_kernel', values=[3, 5]),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))

    model.add(layers.Dense(num_classes, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='helloworld'
)

# Perform the hyperparameter search
tuner.search(train_generator, epochs=10, validation_data=validation_generator)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)


We have plotted the results we have got in the CNN model using plotly express.

# 1. Load Image

In [ ]:
# Example usage:
# Read an image using OpenCV
image_path = 'train/basketball/001.jpg'
original_image = cv2.imread(image_path,cv2.COLOR_BGR2RGB)

plt.imshow(original_image),plt.axis('off')
plt.show()


Image is blue because OpenCV by default print an image in BGR format while Matplotlib does in RGB format

# 2. Resize image

# 3. Normalization

Normalizing imagee for thw whole trainset.

In [ ]:

def normalize_image(image):
    # Convert image to floating-point format
    normalized_image = image.astype('float32')

    # Normalize pixel values to the range [0, 1]
    normalized_image /= 255.0

    return normalized_image

def process_images_in_directory(directory_path):
    # List all files in the directory
    file_list = os.listdir(directory_path)

    for file_name in file_list:
        # Construct the full path to the image file
        image_path = os.path.join(directory_path, file_name)

        try:
            # Read the image
            original_image = cv2.imread(image_path)

            # Ensure the image is not None
            if original_image is not None:
                # Normalize the image
                normalized_image = normalize_image(original_image)

                # Display the original and normalized images (optional)
                cv2.imshow('Original Image', original_image)
                cv2.imshow('Normalized Image', normalized_image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

                # Save the normalized image back to the directory (optional)
                normalized_image_path = os.path.join(directory_path, f'normalized_{file_name}')
                cv2.imwrite(normalized_image_path, (normalized_image * 255).astype(np.uint8))
            else:
                print(f"Error: Unable to read the image from {image_path}")
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")

# Example: Process images in the training directory
train_directory = '/content/dataset/train'
process_images_in_directory(train_directory)


This is for single image

In [ ]:
def normalize_image(image):
    # Convert image to floating-point format
    normalized_image = image.astype('float32')

    # Normalize pixel values to the range [0, 1]
    normalized_image /= 255.0

    return normalized_image


In [ ]:
# Ensure the image is not None
if original_image is not None:
    # Normalize the image
    normalized_image = normalize_image(original_image)

    # Display the original and normalized images (optional)
    cv2.imshow('Original Image', original_image)
    cv2.imshow('Normalized Image', normalized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print(f"Error: Unable to read the image from {image_path}")

Check normalized or not?

In [ ]:

# Ensure the image is not None
if original_image is not None:

    height, width, channels = original_image.shape
    print(f"Image shape: {height} x {width} x {channels}")

    # Choose a pixel coordinate (x, y)
    x, y = 100, 50  # Example coordinates, adjust as needed

    # Print pixel values at the chosen coordinate
    pixel_values = original_image[y, x]
    normalized_pixel_values = normalized_image[y, x]
    print(f"Pixel values at ({x}, {y}): {pixel_values}")
    print(f"Normalized Pixel values at ({x}, {y}): {normalized_pixel_values}")

else:
    print(f"Error: Unable to read the image from {image_path}")


# 4. Color Adjustment

Make sure image is in RGB or required color space

In [ ]:

# Ensure the image is not None
if original_image is not None:
    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Display the RGB image using Matplotlib
    plt.imshow(rgb_image)
    plt.title('RGB Image')
    plt.axis('off')  # Optional: Turn off axis labels
    plt.show()
else:
    print(f"Error: Unable to read the image from {image_path}")


# 5. Grayscale Adjustment

# 6. Contrast Adjustment

In [ ]:

# Ensure the image is not None
if original_image is not None:
    # Split the image into individual color channels
    b, g, r = cv2.split(rgb_image)

    # Adjust contrast for each channel (adjust the contrast factor as needed)
    contrast_factor = 1.2
    adjusted_b = cv2.convertScaleAbs(b, alpha=contrast_factor, beta=0)
    adjusted_g = cv2.convertScaleAbs(g, alpha=contrast_factor, beta=0)
    adjusted_r = cv2.convertScaleAbs(r, alpha=contrast_factor, beta=0)

    # Merge the adjusted channels back into an RGB image
    contrast_adjusted_image = cv2.merge([adjusted_b, adjusted_g, adjusted_r])

    # Display the original and contrast-adjusted images
    plt.imshow(rgb_image)
    plt.show()
    plt.imshow(contrast_adjusted_image)
    plt.show()
else:
    print(f"Error: Unable to read the image from {image_path}")


# 7. Image Augmentation

using another folder containing 3 random images from different folders

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image


In [ ]:
# Define the input and output directories
input_dir = 'Example dataset'
output_dir = 'augmented'

In [ ]:
# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize the ImageDataGenerator with desired augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)



In [ ]:
# Loop through all images in the input directory
for filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, filename)

    # Read the image
    img = Image.open(input_path)
    img_array = np.array(img)

    # Reshape the image to (1, height, width, channels) for flow method
    img_array = img_array.reshape((1,) + img_array.shape)

    # Generate augmented images
    i = 0
    for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_dir, save_prefix='aug', save_format='jpg'):
        i += 1
        if i >= 5:  # Generate 5 augmented images per original image
            break


basically increases dataset size, good for reducing overfitting and generalization. Not sure if we should implement it or not.